In [21]:
import os
import numpy as np
import codecs
path_baptiste = "/home/baptiste/Documents/data/train"
path_igor = "C:/Users/Igor/Documents/Master Data Science/Big Data Analytics/Projet/Data/train"
path_sofia = "/Users/Flukmacdesof/data 2/train"



#assumes labelled data ra stored into a positive and negative folder
#returns two lists one with the text per file and another with the corresponding class 
def loadLabeled(path):

	rootdirPOS =path+'/pos'
	rootdirNEG =path+'/neg'
	data=[]
	Class=[]
	count=0
	for subdir, dirs, files in os.walk(rootdirPOS):
		
		for file in files:
			with codecs.open(rootdirPOS+"/"+file, 'r',encoding="utf-8") as content_file:
				content = content_file.read() #assume that there are NO "new line characters"
				data.append(content)
	tmpc1=np.ones(len(data))
	for subdir, dirs, files in os.walk(rootdirNEG):
		
		for file in files:
			with codecs.open(rootdirNEG+"/"+file, 'r',encoding="utf-8") as content_file:
				content = content_file.read() #assume that there are NO "new line characters"
				data.append(content)
	tmpc0=np.zeros(len(data)-len(tmpc1))
	Class=np.concatenate((tmpc1,tmpc0),axis=0)
	return data,Class
#loads unlabelled data	
#returns two lists
#one with the data per file and another with the respective filenames (without the file extension)
def loadUknown(path):
	rootdir=path
	data=[]
	names=[]
	for subdir, dirs, files in os.walk(rootdir):
		for file in files:
			with open(rootdir+"/"+file, 'r', endoding= "utf-8") as content_file:
				content = content_file.read() #assume that there are NO "new line characters"
				data.append(content)
				names.append(file.split(".")[0])
	return data,names

## Data loading

In [22]:
reviews, Class = loadLabeled(path_sofia)

## First data cleaning:
- Remove all the HTML symbols

In [23]:
# Remove HLML signs
HTMLlist = ['<br />']

for idx, review in enumerate(reviews):
    for word in HTMLlist:
        reviews[idx] = review.replace(word,' ')

## Feature creation:
- List punctuation (various form)

In [24]:
excla = [0]*len(reviews)
inter = [0]*len(reviews)
susp = [0]*len(reviews)
for i, review in enumerate(reviews):
    for char in review:
        if char == "?":
            inter[i] += 1
        elif char == "!":
            excla[i] += 1

In [25]:
from nltk.tokenize.casual import TweetTokenizer
ttoken = TweetTokenizer(reduce_len=True)
tokenized_reviews = []

for review in reviews:
    tokenized_reviews.append(ttoken.tokenize(review))

In [26]:
for i, review in enumerate(tokenized_reviews):
    for word in review:
        if word == "...":
            susp[i] += 1

## Feature creation : 
- Length of the review
- Movie mentionned in the review

In [27]:
rev_length = []
rev_word_count = []


for idr,review in enumerate(reviews):
    # Length of the review
    rev_length.append(len(review))
    
    rev_word_count.append(0)
    for word in review:
        rev_word_count[idr]+=1

In [28]:
# The movie list has to be completed
# - Find a list of the movie so as it matches some in the reviews : www.imdb.com

rev_movie = []

movie_list = {'Titanic':9}
for idr,review in enumerate(reviews):   
    # Movie in the review
    movies = []
    for key, value in movie_list.items():
        if key in review:
            movies.append(value)
    rev_movie.append(movies)

In [29]:
new_rev_movie = []

for movies in rev_movie:
    # No movies quoted
    if movies == []:
        new_rev_movie.append(-1000)
    else:
        # Only one movie quoted
        if len(movies) == 1:
            new_rev_movie.append(movies[0])
        # Different movies quoted - TO BE UPGRADED
        else:
            new_rev_movie.append(-1000)
            

## Feature Creation :
- Grade mentionned in the movie

In [30]:
rev_grade = []
# Grade/Mark in the review
for idr,review in enumerate(reviews):
    rev_grade.append([])
    review_split= review.split(" ")

    for idw, word in enumerate(review_split):
        for idx, char in enumerate(word):
                if char == '/':
                    ten_is_there= False
                    if(idx < len(word)-2):
                        if word[idx+1] == '1' and word[idx+2] == '0':
                            ten_is_there=True
                    if(idx== len(word) -1 and idw<len(review_split)-1 and len(review_split[idw+1])>1 ):
                        if(review_split[idw+1][0]=='1' and review_split[idw+1][1]=='0'):
                            ten_is_there=True
                    if(ten_is_there):                  
                        if(idx)>0:
                            rev_grade[idr].append(word[0:idx])
                        else:
                            rev_grade[idr].append(review_split[idw-1])

In [31]:
# function for reviews with one grade

# DO NOT DELETE THE PRINT NOW : The function may be better when used on the test set

# SOMETIMES : 1/10/2015 -> It is a date ! 

def convert_to_real_grade(grade):
    new_grade = -100
    
    ### The grade is a float
    try:
        new_grade = float(grade)
        return float(new_grade)
    
    ### The grade is not a float
    except:
        good = '0123465789'
        numerical_words = {'zero':0, 'one':1, 'two':1, 'three':3, 'four':4, 'five':5, 
                           'six':6, 'seven':7, 'height':8, 'nine':9, 'ten':10}
        
        ## The grade has numerical values at the end 
        if grade[-1] in good:

            ### Read the grade in the string
            one_dot = False
            g_new = ''
            for char in reversed(grade):
                if char in good:
                    g_new = char + g_new
                elif char in '.,' and one_dot == False:
                    g_new  = '.' + g_new
                    one_dot = True
                else:
                    if g_new[0] in '.,':
                        new_grade = g_new[1:]
                    else:
                        new_grade = g_new
                    
        
        
        
        elif (grade[-1] not in good) and (grade.lower() in numerical_words):
            new_grade = numerical_words[grade.lower()]
            
        return float(new_grade)
                
                    

# functions for reviews with more than one grade
def convert_to_real_grade_2(grade):
    new_grades = []
    for i in range(len(grade)):
        new_grade = convert_to_real_grade(grade[i])
        new_grades.append(new_grade)
    #return new_grades
    return -1000

In [32]:
new_rev_grade = []
for idg, grade in enumerate(rev_grade):    
    converted_grade = -1000
    
    if grade != []:
        if len(grade) == 1:
            converted_grade = convert_to_real_grade(grade[0])
        else:
            converted_grade = convert_to_real_grade_2(grade)
    new_rev_grade.append(converted_grade)
        

## Creating new features : Other ideas to try

- Find N-grams where it may start with a CAPITAL (As for the movie names and actor's names)
- Add smileys

In [33]:
happy = [":-)", ":)", ":D", ":o)", ":]", ":3", ":c)", ":>", "=]", "8)", "=)", ":}", ":^)", ":-))", "^^"]
laughing = [":-D", "8-D", "8D", "x-D", "xD", "X-D", "XD", "=-D", "=D", "=-3", "=3", "B^D"]
sad = [">:[", ":-(", ":(", ":-c", ":c", ":-<", ":<", ":-[", ":[", ":{", ";("]
cry = [":'-(", ":'("]
happy_cry = [":'-)", ":')"]
horror = ["D:<", "D:", "D8", "D;", "D=", "DX", "v.v", "D-':"]
surprised = [">:O", ":-O", ":O", ":-o", ":o", "8-0", "O_O", "o-o", "O_o", "o_O", "o_o", "O-O"]
kiss= [":*", ":^*", "( '}{' )"]
wink = [";-)", ";)", "*-)", "*)", ";-]", ";]", ";D", ";^)", ":-"]
tongue = [">:P", ":-P", ":P", "X-P", "x-p", "xp", "XP", ":-p", ":p", "=p", ":-b", ":b", "d:"]
skeptical = [">:\ ".replace(" ", ""), ">:/", ":-/", ":-.", ":/", ":\ ".replace(" ", ""), "=/", "=\ ".replace(" ", ""), ":L", "=L", ":S", ">.<"]
neutral = [":|", ":-|"]
angel = ["O:-)", "0:-3", "0:3", "0:-)", "0:)", "0;^)"]
evil = [">:)", ">;)", ">:-)", "}:-)", "}:)", "3:-)", "3:)"]
high_five = ["o/\o", "^5", ">_>^ ^<_<"]
heart = ["<3"]
broken_hart = ["</3"]
angry = [":@"]
smiley_list = [
happy,
laughing,
sad,
cry,
happy_cry,
horror,
surprised,
kiss,
wink,
tongue,
skeptical,
neutral,
angel,
evil,
high_five,
heart,
broken_hart, 
angry]
smiley_names = [
"happy",
"laughing",
"sad",
"cry",
"happy_cry",
"horror",
"surprised",
"kiss",
"wink",
"tongue",
"skeptical",
"neutral",
"angel",
"evil",
"high_five",
"heart",
"broken_hart", 
"angry"
]

In [34]:
def gen_features_smiley(tokenized_text, smiley_list):
    features_smiley = np.zeros((len(tokenized_text),len(smiley_list)))
    for i, review in enumerate(tokenized_text):
        for w in review :
            if len(w)<2 : 
                pass
            elif len(w)>5:
                pass
            for j, cat in enumerate(smiley_list):
                if w in cat:
                    features_smiley[i,j] = 1
    return features_smiley

In [35]:
features_smiley = gen_features_smiley(tokenized_reviews, smiley_list)

## Second Data Cleaning (After the features creation) :
 - Punctuation
 - Stop Words


In [36]:
# Remove punctuation, lower all characters
# exclude = {',' ,'+', '<', ':', '/', ']', '(', ')', '{', '"', '_', '?', '@', '}', ...}
import string
exclude = set(string.punctuation)
for idx, review in enumerate(reviews):
    reviews[idx] = ''.join([w if w not in exclude else " " for w in review.lower() ])
    
# Remove stop words based on the given list - To be changed depending on the needs
from nltk.corpus import stopwords
stopwords = stopwords.words("english")
for idx, review in enumerate(reviews):
    reviews[idx] = ''.join([w +' ' for w in review.split() if w not in stopwords])

## Third Data Cleaning: 
- Stemmisation

In [37]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [38]:
# Steeming -> Reduce words to their initial mining

for idx, review in enumerate(reviews):
    reviews[idx] = ''.join([stemmer.stem(word)+' ' for word in review.split()])

In [39]:
reviews

[u'bromwel high cartoon comedi ran time program school life teacher 35 year teach profess lead believ bromwel high satir much closer realiti teacher scrambl surviv financi insight student see right pathet teacher pomp petti whole situat remind school knew student saw episod student repeat tri burn school immedi recal high classic line inspector m sack one teacher student welcom bromwel high expect mani adult age think bromwel high far fetch piti isn ',
 u'homeless houseless georg carlin state issu year never plan help street consid human everyth go school work vote matter peopl think homeless lost caus worri thing racism war iraq pressur kid succeed technolog elect inflat worri ll next end street given bet live street month without luxuri home entertain set bathroom pictur wall comput everyth treasur see like homeless goddard bolt lesson mel brook direct star bolt play rich man everyth world decid make bet sissi rival jefferi tambor see live street thirti day without luxuri bolt succee

## Tf - Idf Matrix

#### To be upgraded with new tf and idf functions!

In [40]:
# Features extraction with TF - IDF : get the matrix
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

m = TfidfVectorizer()
tfidf_matrix = m.fit_transform(reviews)

print("Size of the tfidf matrix: ", tfidf_matrix.size)
print(tfidf_matrix.shape)

('Size of the tfidf matrix: ', 2394756)
(25000, 50615)


In [74]:
unique_words=list(set(''.join([review for review in reviews ]).split(' ')))

In [75]:

num_documents = len(reviews)
sliding_window = 2
train_par = True
idf_learned = {}

In [59]:
import networkx as nx

In [76]:
def createGraphFeatures(num_documents,clean_train_documents,unique_words,sliding_window,train_par,idf_learned):
	features = np.zeros((num_documents,len(unique_words)))#where we are going to put the features
	unique_words_len = len(unique_words)
	term_num_docs = {} #dictionay of each word with a count of that word through out the collections
	idf_col = {}#dictionay of each word with the idf of that word
	
	#TO DO:
	#1.idf_col:IDF for the collection
	#	if in training phase compute it
	#	else use the one provided
	#2. term_num_docs : count of the words in the collection
	#	if in training phase populate it
	#	else use the one provided
	if train_par:
		
		#for all documents
		for i in range( 0,num_documents ):
			wordList = clean_train_documents[i].split(None)
			#count word occurrences through the collection (for idf) put the count in term_num_docs
			if len(wordList)>1:
				countWords(wordList,term_num_docs) #TODO: implement this function 
		#TODO: calculate the idf for all words
		for term_x in term_num_docs:
			idf_col[term_x] = np.log10(float(num_documents)/term_num_docs[term_x])            
	# for the testing set
	else:
		#use the existing ones if we are in the test data
		idf_col = idf_learned 
		term_num_docs=unique_words

	print "Creating the graph of words for each document..."
	totalNodes = 0
	totalEdges = 0

	#go over all documents
	for i in range( 0,num_documents ):
		wordList = clean_train_documents[i].split(None)
		docLen = len(wordList)
		#the graph
		dG = nx.Graph()

		if len(wordList)>1:
			populateGraph(wordList,dG,sliding_window)
			dG.remove_edges_from(dG.selfloop_edges())
			centrality = nx.degree_centrality(dG) #dictionary of centralities (node:degree)

			totalNodes += dG.number_of_nodes()
			totalEdges += dG.number_of_edges()

			
			#TODO : implement comments bellow
			# for all nodes
				#If they are in the desired features
					#compute the TW-IDF score and put it in features[i,unique_words.index(g)]
			for k, node_term in enumerate(dG.nodes()):
				if node_term in idf_col:
					features[i,unique_words.index(node_term)] = centrality[node_term] * idf_col[node_term]

	if train_par:
		nodes_ret=term_num_docs.keys()
		#print "Percentage of features kept:"+str(feature_reduction)
		print "Average number of nodes:"+str(float(totalNodes)/num_documents)
		print "Average number of edges:"+str(float(totalEdges)/num_documents)
	else:
		nodes_ret=term_num_docs
	#return 1: features, 2: idf values (for the test data), 3: the list of terms 
	return features, idf_col, nodes_ret

In [77]:
def populateGraph(wordList,dG,sliding_window):
	#TODO: implement this function
	#For each position/word in the word list:
		#add the -new- word in the graph
		#for all words -forward- within the window size
			#add new words as new nodes 
			#add edges among all word within the window
	for k, word in enumerate(wordList):
		if not dG.has_node(word):
			dG.add_node(word)
		tempW=sliding_window
		if k+sliding_window > len(wordList):
			tempW=len(wordList) - k
		for j in xrange(1, tempW):
			next_word = wordList[k+j]
			dG.add_edge(word, next_word)

In [78]:
def countWords(wordList,term_num_docs):
	found = set()
	#TODO: implement this function
	#add the terms from the wordlist to the term_num_docs dictionary or increase its count
	for k, word in enumerate(wordList):
		if word not in found:
			found.add(word)
			if word in term_num_docs:
				term_num_docs[word] +=1
			else :
				term_num_docs[word] = 1

In [ ]:
%pdb

In [ ]:
tw_idf = createGraphFeatures(num_documents,reviews,unique_words,sliding_window,train_par,idf_learned)

## Adding the new features to the Tf-Idf matrix
#### New features are : 
- Number of exclamation point
- Number of interrogation point
- Number of suspension point
- Review length
- Number of word (word_count)
- Movie mentionned
- Grade mentionned

- Smileys (How to deal with them?)

In [ ]:
import scipy

def csr_vappend(a,b):
    """ Takes in 2 csr_matrices and appends the second one to the bottom of the first one. 
    Much faster than scipy.sparse.vstack but assumes the type to be csr and overwrites
    the first matrix instead of copying it. The data, indices, and indptr still get copied."""

    a.data = np.hstack((a.data,b.data))
    a.indices = np.hstack((a.indices,b.indices))
    a.indptr = np.hstack((a.indptr,(b.indptr + a.nnz)[1:]))
    a._shape = (a.shape[0]+b.shape[0],b.shape[1])
    return a

In [ ]:
excla = scipy.sparse.csr_matrix(excla)
inter = scipy.sparse.csr_matrix(inter)
susp = scipy.sparse.csr_matrix(susp)
rev_length = scipy.sparse.csr_matrix(rev_length)
rev_word_count = scipy.sparse.csr_matrix(rev_word_count)
rev_grade = scipy.sparse.csr_matrix(new_rev_grade)
rev_movie = scipy.sparse.csr_matrix(new_rev_movie)
rev_smiley = scipy.sparse.csr_matrix(features_smiley)

In [ ]:
csr_vappend(tfidf_matrix, excla)
csr_vappend(tfidf_matrix, inter)
csr_vappend(tfidf_matrix, susp)
csr_vappend(tfidf_matrix,rev_length)
csr_vappend(tfidf_matrix,rev_word_count)
csr_vappend(tfidf_matrix,rev_grade)
csr_vappend(tfidf_matrix,rev_movie)
csr_vappend(tfidf_matrix, rev_smiley)

##  CSV Creation:

Create 5 csv : train_train.csv, train_test.csv, y_train_train.csv, y_train_test.csv, test.csv

In [ ]:
# Split the tf-idf matrix into two data sets to process the cross validation : training and test set
from sklearn.cross_validation import train_test_split

data_train, data_test, label_train, label_test = train_test_split(tfidf_matrix, Class, test_size = 0.3, random_state = 42)

In [ ]:
print tfidf_matrix.shape
Class.shape

##### 

In [ ]:

import csv

def save_sparse_csr(filename, array):
    np.savez(filename, data = array.data, indices = array.indices, 
             indptr = array.indptr, shape = array.shape)
    
def load_sparse_csr(filename):
    loader = np.load(filename)
    return scipy.sparse.csr_matrix(( loader['data'], loader['indices'], loader['indptr']),
                     shape = loader['shape'])

def save_csv(filename, array):
    with open(filename, 'wb') as csvfile:
        writer = csv.writer(csvfile, delimiter = '\n')
        writer.writerow(array)
        
def load_csv(filename):
    with open(filename, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter = '\n')
        array = [float(row[0]) for row in reader]
        return array

In [ ]:
save_sparse_csr('data_train', data_train)
save_sparse_csr('data_test', data_test)
save_csv('label_train.csv', label_train)
save_csv('label_test.csv', label_test)